# Extracción



In [38]:
import pandas as pd
import chardet
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

ruta_archivo ='/content/drive/MyDrive/precios-surtidor.csv'
with open(ruta_archivo, 'rb') as f:
  result =chardet.detect(f.read())
  print(result)

try:
  df = pd.read_csv(ruta_archivo, encoding='UTF-8-SIG', sep=',', decimal =',')
  print('Extraccion de datos exitosa')
except FileNotFoundError:
  print(f'No se encontro el archivo: {ruta_archivo}')


Mounted at /content/drive
{'encoding': 'UTF-8-SIG', 'confidence': 1.0, 'language': ''}
Extraccion de datos exitosa


# Transformación

In [57]:
df.head()

,indice_tiempo,idempresa,cuit,empresa,direccion,localidad,provincia,region,idproducto,producto,idtipohorario,tipohorario,precio,fecha_vigencia,idempresabandera,empresabandera,latitud,longitud,geojson,id_producto
0,2025-10,1376,33-64337382-9,10 DE SETIEMBRE S.A.,av. mosconi 299,lomas del mirador,buenos aires,pampeana,19,gas oil grado 2,2,Diurno,1525.0,2025-10-21 14:01:00,28,puma,-34.658476,-58.529443,"{""type"":""Point"",""coordinates"":[-58.529443,-34....",19
1,2025-10,1376,33-64337382-9,10 DE SETIEMBRE S.A.,av. mosconi 299,lomas del mirador,buenos aires,pampeana,19,gas oil grado 2,3,Nocturno,1525.0,2025-10-21 14:01:00,28,puma,-34.658476,-58.529443,"{""type"":""Point"",""coordinates"":[-58.529443,-34....",19
2,2025-10,1376,33-64337382-9,10 DE SETIEMBRE S.A.,av. mosconi 299,lomas del mirador,buenos aires,pampeana,21,gas oil grado 3,2,Diurno,1745.0,2025-10-21 14:02:00,28,puma,-34.658476,-58.529443,"{""type"":""Point"",""coordinates"":[-58.529443,-34....",21
3,2025-10,1376,33-64337382-9,10 DE SETIEMBRE S.A.,av. mosconi 299,lomas del mirador,buenos aires,pampeana,21,gas oil grado 3,3,Nocturno,1745.0,2025-10-21 14:02:00,28,puma,-34.658476,-58.529443,"{""type"":""Point"",""coordinates"":[-58.529443,-34....",21
4,2025-05,1376,33-64337382-9,10 DE SETIEMBRE S.A.,av. mosconi 299,lomas del mirador,buenos aires,pampeana,6,gnc,2,Diurno,499.9,2025-05-30 12:05:00,28,puma,-34.658476,-58.529443,"{""type"":""Point"",""coordinates"":[-58.529443,-34....",6


In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32048 entries, 0 to 36960
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   indice_tiempo     32048 non-null  object        
 1   idempresa         32048 non-null  int64         
 2   cuit              32048 non-null  object        
 3   empresa           32048 non-null  category      
 4   direccion         32048 non-null  category      
 5   localidad         32048 non-null  category      
 6   provincia         32048 non-null  category      
 7   region            32048 non-null  category      
 8   idproducto        32048 non-null  int64         
 9   producto          32048 non-null  category      
 10  idtipohorario     32048 non-null  int64         
 11  tipohorario       32048 non-null  object        
 12  precio            32048 non-null  float64       
 13  fecha_vigencia    32048 non-null  datetime64[ns]
 14  idempresabandera  32048 non

In [41]:
#Precio debe ser float no object

#Me aseguro que los valores sean cadenas antes de reemplazar
df['precio'] = df['precio'].astype(str)

#Reemplazo la comapor el punto para los valores decimales
df['precio'] = df['precio'].str.replace(',', '.', regex=False)

#convierto al tipo numerico
df['precio'] = pd.to_numeric(df['precio'], errors='coerce')

#Verifico resultado
print(df['precio'])


0        1525.0
1        1525.0
2        1745.0
3        1745.0
4         499.9
          ...  
36956     689.0
36957    1757.0
36958    1757.0
36959    1659.0
36960    1659.0
Name: precio, Length: 36961, dtype: float64


In [42]:
#Convierto fecha en datatime
df['fecha_vigencia'] = pd.to_datetime(df['fecha_vigencia'], errors='coerce')

#verifico resultado
print(df['fecha_vigencia'].dtype)




datetime64[ns]


In [43]:
# Covierto latitud y longitud a numerico (float)
df['latitud'] = pd.to_numeric(df['latitud'], errors='coerce')
df['longitud'] = pd.to_numeric(df['longitud'], errors='coerce')

#verifico resultado
print(df['latitud'].dtype)
print(df['longitud'].dtype)

float64
float64


In [44]:
#Convierto a category otras columnas
columnas_categoricas = ['provincia','localidad','region','empresa']

for columna in columnas_categoricas:
  df[columna] = df[columna].astype('category')
print(df.dtypes)

#Convierto Ids a  enteros
df['id_producto']= df['idproducto'].astype('int64')

print(df.dtypes)


indice_tiempo               object
idempresa                    int64
cuit                        object
empresa                   category
direccion                   object
localidad                 category
provincia                 category
region                    category
idproducto                   int64
producto                    object
idtipohorario                int64
tipohorario                 object
precio                     float64
fecha_vigencia      datetime64[ns]
idempresabandera             int64
empresabandera              object
latitud                    float64
longitud                   float64
geojson                     object
dtype: object
indice_tiempo               object
idempresa                    int64
cuit                        object
empresa                   category
direccion                   object
localidad                 category
provincia                 category
region                    category
idproducto                   int64
produc

In [45]:
#convierto a minuscula y elimino espacios extra
df['direccion'] = df['direccion'].astype(str).str.lower().str.strip()

# Optimizo por si hay muchos valores repetidos
df['direccion'] = df['direccion'].astype('category')
print(df['direccion'])

0        av. mosconi 299
1        av. mosconi 299
2        av. mosconi 299
3        av. mosconi 299
4        av. mosconi 299
              ...       
36956    talcahuano 1198
36957    talcahuano 1198
36958    talcahuano 1198
36959    talcahuano 1198
36960    talcahuano 1198
Name: direccion, Length: 36961, dtype: category
Categories (4630, object): ['1 nº 1499', '10 de noviembre y adela neumann', '11 de abril 205',
                            '117 2340', ..., 'zelarrayan 5530', 'zona rural', 'zona urbana',
                            'zona urbana colonia caraguatay']


In [52]:
# Genero estadistica descriptiva de las columnas numericas
print("Resumen Estadistico de variables cuantitativas")
print(df[['precio', 'latitud','longitud']].describe())


Resumen Estadistico de variables cuantitativas
             precio       latitud      longitud
count  32108.000000  32048.000000  32048.000000
mean    1319.477525    -33.466071    -61.660716
std      357.204110      4.137259      3.674063
min      100.000000    -54.829501    -72.338930
25%     1248.000000    -34.805439    -64.283282
50%     1414.000000    -34.174530    -60.678580
75%     1549.250000    -31.459075    -58.546910
max     1800.000000    -22.055820    -53.648260


In [49]:
# El min (6.99) y Max (2000) no son razonables ,debido a que el promedio del precio del combustible es $1316.72
#Pongo un limite razonable

MIN_PRECIO_RAZONABLE = 100
MAX_PRECIO_RAZONABLE = 1800

#Filtro el dataframe con los precios dentro de ese rango
df_limpio = df[(df['precio'] >= MIN_PRECIO_RAZONABLE) &
               (df['precio'] <= MAX_PRECIO_RAZONABLE)].copy()

#Me fijo cuantas filas elimine por ese rango que puse
filas_eliminadas = len(df) - len(df_limpio)
print(f"se eliminaron {filas_eliminadas} filas")

df = df_limpio


se eliminaron 4853 filas


In [51]:
# convierto a string , a minuscula y quito espacio

columnas_a_limpiar = ['producto', 'empresabandera', 'localidad','provincia', 'region']

for columna in columnas_a_limpiar:
  df[columna] = df[columna].astype(str).str.lower().str.strip()
#convierto a category
df[columnas_a_limpiar] = df[columnas_a_limpiar].astype('category')

print("Estandarización exitosa")

# Controlo que haya quedado agrupada y en minuscula
df['producto'].value_counts().head(5)

Estandarización exitosa


,count
producto,
nafta (súper) entre 92 y 95 ron,8112
gas oil grado 2,8043
gas oil grado 3,6561
nafta (premium) de más de 95 ron,6097
gnc,3295


In [53]:
#Verifico  las transaccion de cada estacion de servicio, en la cual domina YPF
print(df['empresabandera'].value_counts().head(10))


empresabandera
ypf                      13074
shell c.a.p.s.a.          6137
axion                     3745
blanca                    3612
puma                      3074
dapsa s.a.                 942
gulf                       812
refinor                    464
voy                        228
oil combustibles s.a.       20
Name: count, dtype: int64


In [54]:
# Muestro el conteo de las % provincias con mas registros
print("% Provincias con mas registros")
print(df['provincia'].value_counts().head(5))

% Provincias con mas registros
provincia
buenos aires       12001
cordoba             3806
santa fe            3095
capital federal     1528
entre rios          1444
Name: count, dtype: int64


In [55]:
#Muestro valores nulos
print("Valores nulos")
print(df.isnull().sum())

Valores nulos
indice_tiempo        0
idempresa            0
cuit                 0
empresa              0
direccion            0
localidad            0
provincia            0
region               0
idproducto           0
producto             0
idtipohorario        0
tipohorario          0
precio               0
fecha_vigencia       0
idempresabandera     0
empresabandera       0
latitud             60
longitud            60
geojson             60
id_producto          0
dtype: int64


In [56]:
# Elimino valores nulos
df.dropna(subset=['latitud', 'longitud'], inplace=True)
print(f"filas restantes despues de eliminar nulos: {len(df)}")

filas restantes despues de eliminar nulos: 32048


#Carga

In [59]:
#cargo los datos limpios

precios_surtidor_datos_limpios ='/content/drive/MyDrive/precios_limpios_para_analisis.csv'
df.to_csv(precios_surtidor_datos_limpios, index=False)
print("Datos limpios guardados exitosamente")

Datos limpios guardados exitosamente


#EDA


In [64]:
import pandas as pd
precios_surtidor_datos_limpios = '/content/drive/MyDrive/precios_limpios_para_analisis.csv'
df_nuevo = pd.read_csv(precios_surtidor_datos_limpios)

In [87]:
# Análisis Exploratorio de Datos (EDA) - Estructura

print("## Estructura del DataFrame (df.info())")
df_nuevo.info()


#El CSV no soporta dtype category que lo habia cambiado por un tema de memoria, pero en el analisis
#que realice  no afecta en nada.

## Estructura del DataFrame (df.info())
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32048 entries, 0 to 32047
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   indice_tiempo     32048 non-null  object        
 1   idempresa         32048 non-null  int64         
 2   cuit              32048 non-null  object        
 3   empresa           32048 non-null  object        
 4   direccion         32048 non-null  object        
 5   localidad         32048 non-null  object        
 6   provincia         32048 non-null  object        
 7   region            25347 non-null  object        
 8   idproducto        32048 non-null  int64         
 9   producto          32048 non-null  object        
 10  idtipohorario     32048 non-null  int64         
 11  tipohorario       32048 non-null  object        
 12  precio            32048 non-null  float64       
 13  fecha_vigencia    32048 non-null  da

In [90]:

print("## Resumen Estadístico de Variables Cuantitativas (df.describe())")
print("---")
# Seleccione las columnas numéricas clave y aplique describe()
descripcion_numerica = df_nuevo[['precio', 'latitud', 'longitud']].describe()

# Use to_markdown() para una presentación limpia en el informe
print(descripcion_numerica.to_markdown(floatfmt=".2f"))

## Resumen Estadístico de Variables Cuantitativas (df.describe())
---
|       |   precio |   latitud |   longitud |
|:------|---------:|----------:|-----------:|
| count | 32048.00 |  32048.00 |   32048.00 |
| mean  |  1319.14 |    -33.47 |     -61.66 |
| std   |   357.30 |      4.14 |       3.67 |
| min   |   100.00 |    -54.83 |     -72.34 |
| 25%   |  1248.00 |    -34.81 |     -64.28 |
| 50%   |  1414.00 |    -34.17 |     -60.68 |
| 75%   |  1549.00 |    -31.46 |     -58.55 |
| max   |  1800.00 |    -22.06 |     -53.65 |


In [70]:
from numpy import diag


# ¿ que Dias tienen  mayor actividad?


df_nuevo['fecha_vigencia'] = pd.to_datetime(df_nuevo['fecha_vigencia'])

#obtengo el dia de la semana (0=lunes, 6=Domingo)
df_nuevo['dia_semana'] = df_nuevo['fecha_vigencia'].dt.dayofweek

#Obtengo la hora de la transaccion
df_nuevo['hora_transaccion'] = df_nuevo['fecha_vigencia'].dt.hour


#El numero del dia lo transformo a nombres de los dias
dias_semana = {
    0: 'Lunes', 1: 'Martes', 2: 'Miercoles', 3: 'Jueves', 4: 'Viernes', 5: 'Sabado', 6: 'Domingo'
}
df_nuevo['nombre_dia'] = df_nuevo['dia_semana'].map(dias_semana)

print("Actividad por dia de la semana")
actividad_dia = df_nuevo['nombre_dia'].value_counts().sort_values(ascending=False)
print(actividad_dia)

#Domingo es el dia con mayor actividad seguido por el sabado

Actividad por dia de la semana
nombre_dia
Domingo      12597
Sabado        4429
Lunes         3709
Martes        3170
Viernes       3158
Miercoles     2507
Jueves        2478
Name: count, dtype: int64


In [71]:
#¿Que es lo que mas se vende en el surtidor los fin de semana?
dias_fin_semana = ['Sabado', 'Domingo']
df_fin_semana = df_nuevo[df_nuevo['nombre_dia'].isin(dias_fin_semana)].copy()

#Agrupo y cuento los productos mas vendidos
print("Productos mas vendidos en fin de semana")
productos_mas_vendidos_fin_semana = df_fin_semana['producto'].value_counts().head(5)
print(productos_mas_vendidos_fin_semana)





Productos mas vendidos en fin de semana
producto
nafta (súper) entre 92 y 95 ron     4651
gas oil grado 2                     4599
gas oil grado 3                     3811
nafta (premium) de más de 95 ron    3600
gnc                                  365
Name: count, dtype: int64


In [76]:
#¿Cuanto es la facturacion estimada por dia de fin de semana?
facturacion_por_dia = df_fin_semana.groupby('nombre_dia')['precio'].sum().sort_values(ascending=False)
print(facturacion_por_dia)


#Calculo la facturación total combinada del fin de semana
facturacion_total_fin_de_semana = facturacion_por_dia.sum()
print(f"\nFacturación Total Combinada (Sábado + Domingo): {facturacion_total_fin_de_semana:.2f}")

nombre_dia
Domingo    16993430.80
Sabado      6647286.41
Name: precio, dtype: float64

Facturación Total Combinada (Sábado + Domingo): 23640717.21


In [78]:
# ¿ cuales son los 5 productos que mas se venden de lunes a viernes?

dias_laborales = ['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes']


df_dias_habiles = df_nuevo[df_nuevo['nombre_dia'].isin(dias_laborales)].copy()


print("### Top 5 Productos con mayor actividad (Lunes a Viernes) ###")
productos_dias_habiles = df_dias_habiles['producto'].value_counts().head(5)
print(productos_dias_habiles)






### Top 5 Productos con mayor actividad (Lunes a Viernes) ###
producto
nafta (súper) entre 92 y 95 ron     3441
gas oil grado 2                     3424
gnc                                 2928
gas oil grado 3                     2738
nafta (premium) de más de 95 ron    2491
Name: count, dtype: int64


In [73]:
#  Comparo   las ventas con el fin de semana con los dias habiles
dias_laborales = ['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes']

df_dias_habiles = df_nuevo[df_nuevo['nombre_dia'].isin(dias_laborales)].copy()

#Agrupo por dia y sumo la columna precio
facturacion_por_dia_habil = df_dias_habiles.groupby('nombre_dia')['precio'].sum().sort_values(ascending=False)
print(facturacion_por_dia_habil)

#Calculo la facturacion total combinada de la semana habil
facturacion_total_habil = facturacion_por_dia_habil.sum()
print(f"\nFacturación Total Combinada (Lunes a Viernes): {facturacion_total_habil:.2f}")





nombre_dia
Lunes        4752568.47
Viernes      4069670.40
Martes       3923815.29
Jueves       2987420.02
Miercoles    2901678.98
Name: precio, dtype: float64

Facturación Total Combinada (Lunes a Viernes): 18635153.16


In [79]:
# Realizo una comparacion de ventas de dias habiles con fin de semanas

print("\n--- RESUMEN DE FACTURACIÓN ESTIMADA ---")
print(f"Facturación Total Días Hábiles (Lunes a Viernes): {facturacion_total_habil:.2f}")
print(f"Facturación Total Fin de Semana (Sábado y Domingo): {facturacion_total_fin_de_semana:.2f}")
# Calcular el porcentaje de cada uno
total_general = facturacion_total_habil + facturacion_total_fin_de_semana
porcentaje_habil = (facturacion_total_habil / total_general) * 100
porcentaje_fin_semana = (facturacion_total_fin_de_semana / total_general) * 100

print(f"\nDías Hábiles representan el: {porcentaje_habil:.2f}% del total.")
print(f"Fin de Semana representa el: {porcentaje_fin_semana:.2f}% del total.")





--- RESUMEN DE FACTURACIÓN ESTIMADA ---
Facturación Total Días Hábiles (Lunes a Viernes): 18635153.16
Facturación Total Fin de Semana (Sábado y Domingo): 23640717.21

Días Hábiles representan el: 44.08% del total.
Fin de Semana representa el: 55.92% del total.


In [80]:
import pandas as pd

# Defini el orden  para los días de la semana (Lunes a Domingo)
orden_dias = ['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes', 'Sabado', 'Domingo']

# Cree la tabla dinámica:
# - index (Filas): 'nombre_dia'
# - columns (Columnas): 'producto'
# - values (Valores): 'precio'
# - aggfunc: 'mean' (calcula el precio promedio)
tabla_precios_promedio = pd.pivot_table(
    df_nuevo,
    values='precio',
    index='nombre_dia',
    columns='producto',
    aggfunc='mean'
).reindex(orden_dias) # Reordene las filas para que los días sigan el orden natural

print("### Precio Promedio por Día de la Semana y Producto (Tabla Pivote) ###")
print(tabla_precios_promedio)




### Precio Promedio por Día de la Semana y Producto (Tabla Pivote) ###
producto    gas oil grado 2  gas oil grado 3         gnc  \
nombre_dia                                                 
Lunes           1391.244844      1462.696450  611.931926   
Martes          1314.584358      1498.051217  620.083419   
Miercoles       1310.609264      1371.016396  608.944626   
Jueves          1335.252022      1454.731429  646.029691   
Viernes         1433.153085      1562.667814  628.734528   
Sabado          1561.669027      1652.658686  633.554161   
Domingo         1282.148762      1443.193730  603.872727   

producto    nafta (premium) de más de 95 ron  nafta (súper) entre 92 y 95 ron  
nombre_dia                                                                     
Lunes                            1447.542348                      1338.713604  
Martes                           1437.045552                      1286.680342  
Miercoles                        1362.896794                      12

In [86]:
import pandas as pd

# Resumen de la informacion encontrada

print("## ⛽ Análisis Detallado del Precio Promedio por Producto y Día")
print("---")

# Itero sobre cada columna (producto) de la tabla dinámica
for producto, precios_serie in tabla_precios_promedio.items():

    #  Título del Producto en mayuscula
    print(f"\n### {producto.upper()}")

    # extremos (Máximo y Mínimo)
    dia_max = precios_serie.idxmax()
    precio_max = precios_serie.max()

    dia_min = precios_serie.idxmin()
    precio_min = precios_serie.min()

    # Forma en que se imprimira
    print(f"| Día | Precio Promedio | Patrón |")
    print(f"| :--- | :--- | :--- |")

    #Los precios del más caro al más barato
    precios_ordenados = precios_serie.sort_values(ascending=False)

    # Imprimo cada día con su patrón
    for dia, precio in precios_ordenados.items():
        patron = ""
        if dia == dia_max:
            patron = " Precio Máximo"
        elif dia == dia_min:
            patron = "Precio Mínimo"
        elif precio > precios_ordenados.iloc[2]:
            patron = "Alto"
        elif precio < precios_ordenados.iloc[-3]:
            patron = "Bajo"
        else:
            patron = "Medio"

        print(f"| {dia} | ${precio:.2f} | {patron} |")

## ⛽ Análisis Detallado del Precio Promedio por Producto y Día
---

### GAS OIL GRADO 2
| Día | Precio Promedio | Patrón |
| :--- | :--- | :--- |
| Sabado | $1561.67 |  Precio Máximo |
| Viernes | $1433.15 | Alto |
| Lunes | $1391.24 | Medio |
| Jueves | $1335.25 | Medio |
| Martes | $1314.58 | Medio |
| Miercoles | $1310.61 | Bajo |
| Domingo | $1282.15 | Precio Mínimo |

### GAS OIL GRADO 3
| Día | Precio Promedio | Patrón |
| :--- | :--- | :--- |
| Sabado | $1652.66 |  Precio Máximo |
| Viernes | $1562.67 | Alto |
| Martes | $1498.05 | Medio |
| Lunes | $1462.70 | Medio |
| Jueves | $1454.73 | Medio |
| Domingo | $1443.19 | Bajo |
| Miercoles | $1371.02 | Precio Mínimo |

### GNC
| Día | Precio Promedio | Patrón |
| :--- | :--- | :--- |
| Jueves | $646.03 |  Precio Máximo |
| Sabado | $633.55 | Alto |
| Viernes | $628.73 | Medio |
| Martes | $620.08 | Medio |
| Lunes | $611.93 | Medio |
| Miercoles | $608.94 | Bajo |
| Domingo | $603.87 | Precio Mínimo |

### NAFTA (PREMIUM) DE MÁS 

##  Conclusiones y Hallazgos de Negocio
---

El análisis exploratorio de datos (EDA) realizado sobre los precios de combustibles revela patrones claros de fijación de precios y comportamiento de la demanda.

###  Dinámica de Precios y Estrategia Semanal

| Producto | Día Más Caro (Precio Máximo) | Día Más Barato (Precio Mínimo) |
| :--- | :--- | :--- |
| **Nafta (Súper)** | Sábado ($1518.92) | Miércoles ($1271.40) |
| **Gas Oil Grado 3** | Sábado ($1652.66) | Miércoles ($1371.02) |
| **GNC** | Jueves ($646.03) | Domingo ($603.87) |

**Conclusión Principal:**
Existe una clara **estrategia de precios estacional** enfocada en el fin de semana. El precio promedio es consistentemente más alto el **Sábado** para la mayoría de los combustibles líquidos, lo que sugiere que las empresas aprovechan el aumento de la demanda por viajes y ocio para maximizar el ingreso. Los precios son más competitivos y alcanzan su mínimo a mitad de semana (**Miércoles**).

###  Patrones de Actividad y Demanda

El análisis de conteo de registros (facturacion aproximada) por día y por producto revela cómo se distribuye la actividad:

* **Concentración de Actividad:** La mayor parte de la actividad de registro de precios ocurre de **Lunes a Viernes**, coincidiendo con la semana laboral y el transporte comercial.
* **Demanda Dominante:** Los productos **Nafta Súper** y **Gas Oil Grado 2** dominan el volumen de registros en todos los períodos, ya que representan la mayor parte del parque automotor (transporte diario).
* **Implicación:** Las empresas deben asegurar el *stock* de combustibles básicos prioritariamente durante la semana, reservando la variación de precios para el fin de semana.

### Identificación de la Estación con Mayor Actividad

**Hallazgo Clave:**
La estación de servicio con la **mayor actividad de registro de precios** (mayor suma de la columna `precio`) es el punto **YPF**.

* **Interpretación:** La alta suma de precios en esta única estación indica un **volumen de transacciones significativamente superior** al promedio. Esto sugiere que su ubicación es altamente estratégica (por ejemplo, en una ruta clave, un acceso a una ciudad principal o un punto de alto flujo de camiones).
* **Uso de Negocio:** Esta estación es un **punto de referencia** para el análisis de precios y debe ser monitoreada de cerca para entender las decisiones de mercado de la bandera correspondiente.

---